In [16]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import csv
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

# WebDriver başlatılıyor
browser = webdriver.Chrome()
browser.get('https://www.trendyol.com/apple-elektronik-x-b101470-c104024?pi=31')
time.sleep(3)

# Sayfa kaydırma işlemi için son yükseklik değeri ve kaydırma oranları
last_height = browser.execute_script("return document.body.scrollHeight")
scroll_percentages = [0.75, 0.76, 0.77, 0.78, 0.79, 0.80, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]

# Sayfa kaydırma döngüsü
for percentage in scroll_percentages:
    while True:
        current_height = browser.execute_script("return document.body.scrollHeight")
        browser.execute_script(f"window.scrollTo(0, {current_height * percentage});")
        time.sleep(0.2)
        new_height = browser.execute_script("return document.body.scrollHeight")
        
        if new_height == current_height:
            break
        
        current_height = new_height

    if percentage == 0.99:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.4)
        final_height = browser.execute_script("return document.body.scrollHeight")
        
        if final_height == new_height:
            print("Artık kaydırma yapılamıyor. Program kapanıyor.")
            break

# Sayfa kaynağını al ve BeautifulSoup ile ayrıştır
page_source = browser.page_source
trendyol_soup = BeautifulSoup(page_source, "lxml")

# Ürün bilgilerini bulma
urunler = trendyol_soup.find_all("div", attrs={"class": "p-card-wrppr with-campaign-view"})
urun_bilgileri = []

for urun in urunler:
    urun_info = {}
    urun_info['title'] = urun.get('title')

    # Ürün bağlantısı
    urun_info['link'] = 'https://www.trendyol.com' + urun.find('a')['href'] if urun.find('a') else None

    # Ürün resmi
    resim = urun.find('img', class_='p-card-img')
    urun_info['image'] = resim['src'] if resim else None

    # Ürün fiyatı
    fiyat = urun.find('div', class_='prc-box-dscntd')
    urun_info['price'] = fiyat.text.strip() if fiyat else None

    # Marka, model ve özellik bilgileri
    marka = urun.find('span', class_='prdct-desc-cntnr-ttl')
    model = urun.find('span', class_='prdct-desc-cntnr-name')
    ozellik = urun.find('div', class_='product-desc-sub-text')

    urun_info['brand'] = marka.text.strip() if marka else None
    urun_info['model'] = model.text.strip() if model else None
    urun_info['ozellik'] = ozellik.text.strip() if ozellik else None

    urun_bilgileri.append(urun_info)

# Ürün bilgilerini ekrana yazdır
for urun in urun_bilgileri:
    print(urun)

# Verileri Excel dosyasına kaydetme
df = pd.DataFrame(urun_bilgileri)
df.to_excel("trendyol_veri.xlsx", index=False, engine='openpyxl')

print(f"Toplam ürün sayısı: {len(urun_bilgileri)}")
print("Veriler trendyol_veri.xlsx dosyasına başarıyla kaydedildi.")


{'title': 'iPhone 13 128 GB Yıldız Işığı Cep Telefonu (Apple Türkiye Garantili)', 'link': '/apple/iphone-13-128-gb-yildiz-isigi-cep-telefonu-apple-turkiye-garantili-p-150059024?boutiqueId=61&merchantId=355135', 'image': 'https://cdn.dsmcdn.com/ty814/product/media/images/20230406/11/319331216/250484948/1/1_org.jpg', 'price': '38.699 TL', 'brand': 'Apple', 'model': 'iPhone 13 128 GB Yıldız', 'ozellik': 'Işığı Cep Telefonu (Apple Türkiye Garantili)'}
{'title': 'Macbook Air M1 Çip 8gb 256gb Ssd Macos 13" Qhd Taşınabilir Bilgisayar Uzay Grisi', 'link': '/apple/macbook-air-m1-cip-8gb-256gb-ssd-macos-13-qhd-tasinabilir-bilgisayar-uzay-grisi-p-68042136?boutiqueId=638145&merchantId=968', 'image': 'https://cdn.dsmcdn.com/ty1018/product/media/images/prod/SPM/PIM/20231019/13/899e8cb7-16a0-3617-bae0-555eacd2e713/1_org.jpg', 'price': '29.499 TL', 'brand': 'Apple', 'model': 'Macbook Air M1 Çip 8gb', 'ozellik': '256gb Ssd Macos 13" Qhd Taşınabilir Bilgisayar Uzay Grisi'}
{'title': 'iPhone 16 128GB Bey